In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Missões

Vou colocando cada missão como markdown e aí geramos um código para resolver cada uma.
Abaixo segue um exemplo com a importação dos dados e quantificação dos nulls.

## Importação da tabela de treino ID

In [ ]:
#carrega o csv para o dataframe
df = pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")
total_nrows_train = len(df)

## Quantificação de NaNs por features

Também precisamos saber o quanto esses valores representam do total. Para isso precisamos dividir esses valores pela quantidade total de casos.

In [ ]:
#a função isnull retorna uma matriz, com true para todos os valores que sao nulos e false para os que nao sao nulos
#aí , nós somamos os valores true por coluna , e com isso conseguimos perceber as colunas que tem mais dados faltantes
df.isnull().sum(axis = 0)/len(df)

## Quantificação de NaN por rows

O valor de rows excluível por NaN na literatura é de 5% do total, então precisamos encontrar uma combinação de features e rows(linhas) que satisfaça esse valor.
Primeiro, precisamos quantificar os NaN por rows.

In [ ]:
 df.isnull().sum(axis = 1)/len(df.columns)

## Rows Incompletas

Em seguida encontramos a quantidade de rows que não estão completas (possuem algum valor NaN 0).

In [ ]:
#aqui estamos verificando se cada row possui pelo menos um valor NaN
#e obtendo a razão de rows que possuem pelo menos um valor NaN
(df.isnull().sum(axis = 1) != 0).sum(axis = 0)/len(df)

 ## Seleçao de colunas

In [ ]:
limiteNan = 0.05
dataframe = df

listaColunasRemovidas = []
listaColunasNaoRemovidas = []

for i in df.columns:
    if(dataframe[i].isnull().sum(axis = 0)/len(dataframe) > limiteNan):
        #insere na lista de colunas a serem removidas
        listaColunasRemovidas.insert(0,i)
    else:
        #insere na lista de colunas a não serem removidas
        listaColunasNaoRemovidas.insert(0,i)

dataframe = dataframe.drop(columns = listaColunasRemovidas)

df = dataframe

## Seleção de Rows


In [ ]:
#aqui calculamos para todo o dataframe, o percentual de rows que possuem algum valor NaN
df.drop(df[(df.isnull().sum(axis = 1) != 0)].index).count()/len(df)
                                                                                      
#agora que , no nosso dataframe garantimos que temos menos de 5% de rows com algum valor NaN
#podemos excluir as rows que tem algum valor NaN
df.drop(df[(df.isnull().sum(axis = 1) != 0)].index,inplace = True)

print(df)



Precisamos checar se o valor de 95% foi atingido. (o teste retornará true, caso isso aconteça)

In [ ]:
(total_nrows_train * 0.95 < len(df))

precisamos checar se não existe nenhum valor nulo na tabela de treino

In [ ]:
#calcula a quantidade de valores nulos na tabela de treino
((df.isnull().sum(axis = 1) != 0).sum(axis = 0) == 0)

## Pré Processamento na tabela de Teste

Utilizaremos o modelo treinado na tabela de treino para predizer a classe fraude na tabela de teste, assim, elas precisam ter as mesmas features. Além disso, em relação ao número de rows, ela precisa representar ao menos 25% do total da tabela de treino, para termos uma amostra representativa.
Precisamos selecionar as mesmas features que restaram na tabela de treino agora na tabela de teste e checar se a condição anterior será obedecida. Se não for, repita a seleção até conseguir.

naoNanMinimo = 0.95



In [ ]:
df2 = pd.read_csv("../input/ieee-fraud-detection/test_identity.csv")

total_nrows_test = len(df2)

df2 = df2.drop(columns = listaColunasRemovidas)

#agora que , no nosso dataframe garantimos que temos menos de 5% de rows com algum valor NaN
#podemos excluir as rows que tem algum valor NaN
df2.drop(df2[(df2.isnull().sum(axis = 1) != 0)].index,inplace = True)


print(df2)

Fazemos um pequeno teste para saber se a a tabela de testes não possui rows incompletas

In [ ]:
((df2.isnull().sum(axis = 1) != 0).sum(axis = 0) == 0)

Fazemos um pequeno teste , para saber se mais de 5% das rows da tabela de teste foram excluidas

In [ ]:
(total_nrows_train * 0.95 < len(df))

## Exportação dos dados

Para facilitar nossa organização, criaremos Kernels diferentes para cada tarefa básica (Pré Processamento, Análise Exploratória e Modelagem). Assim, precisamos exportar essas novas tabelas para usarmos em outro Kernel.

In [ ]:
df.to_csv('trainID.csv')
df2.to_csv('testID.csv')

## Pré Processamento da tabela de Transações

Agora, criamos outro Kernel para o Pré Processamento da tabela de transações e repetimos todo o processo nela.
Essa tabela é bem maior (591k rows) então pode ser que precise gerenciar a memória RAM (carregando uma feature ou n rows por vez, por exemplo).